In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "2"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton
from ase.optimize import LBFGS

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

from ase.io import read, write

from ase.parallel import world

In [2]:
sdir = '/home/jsmith48/scratch/carbon_nano_tube/all_xyz/'
odir = '/home/jsmith48/scratch/carbon_nano_tube/dft_opt_xyz/'

# Molecule file
molfile = [f for f in os.listdir(sdir) if '.xyz' in f]

C = 0.001 # Optimization convergence

ntdir = '/home/jsmith48/scratch/auto_dhl_al/modeldhl/ANI-1x-DHL-0000/ANI-1x-DHL-0000.0019/'
cns = ntdir + 'train0/rHCNO-5.2R_16-3.5A_a4-8.params'
sae = ntdir + 'train0/sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 8

In [3]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 4)

In [6]:
total_time = 0
for f in molfile:
    # Load molecule
    mol = read(sdir+f)
    
    # Set ANI calculator
    mol.set_calculator(ANIENS(aens))

    spc = mol.get_chemical_symbols()
    print(f,len(spc))

    # Optimize molecule
    start_time = time.time()
    dyn = LBFGS(mol,logfile='LOG.OUT')
    dyn.run(fmax=C,steps=10000)
    print('[ANI Total time:', time.time() - start_time, 'seconds] Total steps: '+str(dyn.nsteps))
    total_time += time.time() - start_time

    # Save optimized mol
    spc = mol.get_chemical_symbols()
    pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)


    hdt.writexyzfile(odir+f.rsplit('.',1)[0]+'_aniopt.xyz', pos, spc)

11_0_ortho1.xyz 1106
[ANI Total time: 3.5684120655059814 seconds] Total steps: 163
5_4_pristine.xyz 994
[ANI Total time: 5.1030378341674805 seconds] Total steps: 253
8_6_ortho2.xyz 1228
[ANI Total time: 23.10005211830139 seconds] Total steps: 1015
9_1_ortho3.xyz 1136
[ANI Total time: 7.85323429107666 seconds] Total steps: 349
16_0_ortho1.xyz 1600
[ANI Total time: 11.005992412567139 seconds] Total steps: 412
6_2_ortho1.xyz 864
[ANI Total time: 8.288365840911865 seconds] Total steps: 404
9_1_ortho2.xyz 1136
[ANI Total time: 6.808588027954102 seconds] Total steps: 308
16_0_ortho2.xyz 1600
[ANI Total time: 10.637757062911987 seconds] Total steps: 399
5_4_ortho2.xyz 1006
[ANI Total time: 6.520658493041992 seconds] Total steps: 298
14_0_ortho3.xyz 1404
[ANI Total time: 11.683601379394531 seconds] Total steps: 467
7_6_ortho3.xyz 1562
[ANI Total time: 19.948963403701782 seconds] Total steps: 745
10_5_ortho3.xyz 1450
[ANI Total time: 7.29061484336853 seconds] Total steps: 292
10_5_ortho2.xyz 14

In [5]:
print(total_time)

526.6713964939117
